<a href="https://colab.research.google.com/github/KevinTheRainmaker/ML_DL_Basics/blob/master/Udacity%3A%20Intro%20to%20TensorFlow%20for%20DL/UTFD_L9C11_Building_a_Basic_Sentiment_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embeddings
- Embeddings are clusters of vectors in multi-dimensional space
- Each vector represents a given word in that space

- [Link: Why Word Embeddings are used in NLP?](https://towardsdatascience.com/why-do-we-use-embeddings-in-nlp-2f20e1b632d2)



## Building a Basic Sentiment Model
To create our embeddings, we'll first use an embedding layer:
- `tf.keras.layers.Embedding`
  - Needs size of tokenized vocabulary
  - How many embedding dimensions to use
  - Input length = max length of sequence
- Output is NOT ready for Dense/fully=connected 
  - Need to be reshaped to work with any fully-connected layers
  - `Flatten()` of `GlobalAveragePooling1D()` layers for a little additional computation that sometimes creates better results
- Follow with any Dense layers, with final output of shape 1

## Entropy
The average quantity of information generated by probability is Entropy.

정보량(Quantity of Information)== '놀람의 정도'

정보량 $I(X) = log({1\over p(x)})$

\* log 취하는 이유: 정보량을 표현하는데에 필요한 최소 resource 표현 가능

ex - 확률 1/8인 사건 > $- log_2({1\over 8}) = 3$ > 최소 3 bit 필요


Entropy $H_p(X) = E[I(X)] = - \Sigma^n_{i=1} p(x_i) log(p(x_i))$

## Cross Entropy

두 개의 확률분포 p와 q에 대해 하나의 event X가 갖는 Quantity of Information.

Cross Entropy $H_{p,q}(X) = - \Sigma^N_{i=1} p(x_i)logq(x_i)$

\* p = true probability / q = 현재 예측모델의 추정값에 대한 분포

### Binary Cross Entropy
In our case, we're only looking at positive vs. negative sentiment, so only a single output node is needed.

Since the result is only binary classification(0 for negetive, 1 for positive), we'll be able to use Binary Cross Entropy.

Binary Cross Entropy는 두 개의 class 중 하나를 예측하는 task에 대한 Cross Entropy의 Special case

<br>

$BCE(x) = - {1\over N} \Sigma^N_{i=1} y_i log(h(x_i;\theta)) + (1-y_i)log(1-h(x_i:\theta))$



## Import classes

In [16]:
%tensorflow_version 2.x

import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np
import pandas as pd

import io

## Get the dataset


In [5]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2021-10-12 10:33:54--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 142.250.148.101, 142.250.148.100, 142.250.148.138, ...
Connecting to drive.google.com (drive.google.com)|142.250.148.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nssqm95a7r943lhujpmnckeeoom117f8/1634034825000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P [following]
--2021-10-12 10:33:54--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nssqm95a7r943lhujpmnckeeoom117f8/1634034825000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 172.217.212.132, 2607:f8b0:4001:c03::84
Connecting to doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)|1

In [6]:
dataset = pd.read_csv('/tmp/sentiment.csv')

sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

## Tokenize the dataset

Tokenize the dataset, including padding and OOV

In [9]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, 
                               padding=padding_type, truncating=trunc_type)

## Review a Sequence

In [10]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

good case excellent value ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Good case Excellent value.


## Train a Basic Sentiment Model with Enbeddings

In [12]:
# Build a basic sentiment network
# Note the embedding layer is first, 
# and the output is only 1 node as it is either 0 or 1 (negative or positive)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 16)           16000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 9606      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [14]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
50/50 [==============================] - 3s 10ms/step - loss: 0.6926 - accuracy: 0.5361 - val_loss: 0.6996 - val_accuracy: 0.4110
Epoch 2/10
50/50 [==============================] - 0s 7ms/step - loss: 0.6874 - accuracy: 0.5229 - val_loss: 0.6993 - val_accuracy: 0.4160
Epoch 3/10
50/50 [==============================] - 0s 7ms/step - loss: 0.6752 - accuracy: 0.6070 - val_loss: 0.6863 - val_accuracy: 0.5263
Epoch 4/10
50/50 [==============================] - 0s 7ms/step - loss: 0.6374 - accuracy: 0.7389 - val_loss: 0.6599 - val_accuracy: 0.6090
Epoch 5/10
50/50 [==============================] - 0s 6ms/step - loss: 0.5574 - accuracy: 0.8154 - val_loss: 0.6135 - val_accuracy: 0.6792
Epoch 6/10
50/50 [==============================] - 0s 6ms/step - loss: 0.4519 - accuracy: 0.8864 - val_loss: 0.5284 - val_accuracy: 0.7444
Epoch 7/10
50/50 [==============================] - 0s 7ms/step - loss: 0.3530 - accuracy: 0.9178 - val_loss: 0.4864 - val_accuracy: 0.7519
Epoch 8/10
50/50 [=

## Get files for visualizing the network

The code below will download two files for visualizing how our network 'sees' the sentiment related to each word.

Head to http://projector.tensorflow.org/ and load these files, then click the "Sphereize" checkbox.

In [15]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(1000, 16)


In [17]:
# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [18]:
# Download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Predicting Sentiment in New Reviews

In [19]:
# Use the model to predict a review   
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(fake_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')              

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

# Try adding reviews of your own
# Add some negative words (such as "not") to the good reviews and see what happens
# For example:
# they gave us free chocolate cake and did not charge us

['I love this phone', 'I hate spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head']

HOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!

I love this phone
[0.9726493]


I hate spaghetti
[0.12117758]


Everything was cold
[0.46837828]


Everything was hot exactly as I wanted
[0.677336]


Everything was green
[0.5081195]


the host seated us immediately
[0.7919122]


they gave us free chocolate cake
[0.7491515]


not sure about the wilted flowers on the table
[0.0269625]


only works when I stand on tippy toes
[0.8105773]


does not work when I stand on my head
[0.01726029]


